# Baseline model for adult data

In this notebook we train a simple model on the adult data that can serve as a counterfactual for what would have happened if we hadn't made any kind of fairness intervention.

In [1]:
import joblib
from pathlib import Path

import pandas as pd
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
)
from helpers.utils import bin_hours_per_week
from helpers.utils import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from helpers.utils import group_box_plots
from sklearn.neural_network import MLPClassifier  # noqa

Directory containing preprocessed data.

In [2]:
artifacts_dir = Path("artifacts")

Load the preprocessed data. Check out the preprocessing notebook for details on how this data was obtained.

In [3]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

train_oh = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val_oh = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test_oh = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

## Training a model to predict salary

We will load a model from disk so that results are reproducible, but commented out here is the code we used to train the model.

In [4]:
model = MLPClassifier(hidden_layer_sizes=(100, 100), early_stopping=True)

model.fit(train_oh.drop(columns="salary"), train_oh.salary)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100))

Load the pretrained model

In [ ]:
# model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

Model accuracy on test set

In [5]:
test_prob = model.predict_proba(test_oh.drop(columns="salary"))[:, 1]
test_pred = test_prob > 0.5
test_accuracy = model.score(test_oh.drop(columns="salary"), test_oh.salary)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Test accuracy: 84.84%


## Demographic parity

We measure demographic parity using both the difference and ratio of acceptance rates. These metrics are implemented for us in the `fairlearn` library.

We also compare the distribution of scores for each sex using box plots of scores.

In [6]:
dpd = demographic_parity_difference(
    test_oh.salary, test_pred, sensitive_features=test_oh.sex,
)
dpr = demographic_parity_ratio(
    test_oh.salary, test_pred, sensitive_features=test_oh.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.218
Demographic parity ratio: 0.272


In [7]:
fig_dp_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    title="Distribution of scores by sex",
    xlabel="Score",
)
fig_dp_by_sex

We calculate similar metrics and produce similar plots for race.

In [8]:
dpd = demographic_parity_difference(
    test_oh.salary, test_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    test_oh.salary, test_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.239
Demographic parity ratio: 0.219


In [9]:
race_names = {
    "amer_indian_eskimo": "American Indian / Eskimo",
    "asian_pac_islander": "Asian / Pacific Islander",
    "black": "Black",
    "other": "Other",
    "white": "White",
}

fig_dp_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    title="Distribution of scores by race",
    xlabel="Score",
)
fig_dp_by_race

## Conditional demographic parity

Distribution by sex and hours worked per week.

In [10]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    test_oh.salary, test_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    test_oh.salary, test_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.191
Conditional demographic parity ratio: 0.460


In [11]:
fig_cdp_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    groups=test_hpw_enum,
    group_names=["0-30", "30-40", "40-50", "50+"],
    title="Distribution of scores by sex and hours worked per week",
    xlabel="Score",
    ylabel="Hours worked per week",
)
fig_cdp_by_sex

Distribution by race and hours worked per week.

In [13]:
cdpd = conditional_demographic_parity_difference(
    test_oh.salary, test_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    test_oh.salary, test_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.298
Conditional demographic parity ratio: 0.113


In [12]:
fig_cdp_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    groups=test_hpw_enum,
    group_names=["0-30", "30-40", "40-50", "50+"],
    title="Distribution of scores by race and hours worked per week",
    xlabel="Score",
    ylabel="Hours worked per week",
)
fig_cdp_by_race

## Equalised odds

To assess equalised odds we compare scores across the outcome classes.

In [14]:
eod = equalized_odds_difference(
    test_oh.salary, test_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    test_oh.salary, test_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.151
Equalised odds ratio: 0.189


In [15]:
fig_eo_by_sex = group_box_plots(
    test_prob,
    test.sex.map(lambda x: "Male" if x else "Female"),
    groups=test.salary,
    group_names=["Low earner", "High earner"],
    title="Distribution of scores by sex for high and low earners",
    xlabel="Score",
    ylabel="High / low earner",
)
fig_eo_by_sex

We do the same, comparing races.

In [16]:
eod = equalized_odds_difference(
    test_oh.salary, test_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    test_oh.salary, test_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.344
Equalised odds ratio: 0.077


In [17]:
fig_eo_by_race = group_box_plots(
    test_prob,
    test.race.map(race_names),
    groups=test.salary,
    group_names=["Low earner", "High earner"],
    title="Distribution of scores by race for high and low earners",
    xlabel="Score",
    ylabel="High / low earner",
)
fig_eo_by_race